# Istari AIAA Hands on Keyboard Event
*Last updated: January 5, 2025 4:45 PM EST*

## About this doc

This notebook demonstrates the Istari platform workflow for a Group 3 expendable UAV (tailless flying wing).
You will modify wing design parameters, run an nTop structural analysis, and verify compliance with system requirements.

**Goal**: Achieve the lowest Structure Weight while meeting all requirements.

In [ ]:
#@title User Data { display-mode: "form" }
!pip install istari-digital-client -q

from google.colab import userdata

#@markdown Your `ISTARI_PAT` should be stored in Colab Secrets (🔑 in sidebar)

ISTARI_ENVIRONMENT_URL = "https://fileservice-v2.stage.istari.app"
ISTARI_PAT = userdata.get('ISTARI_PAT')

#@markdown ---

NTOP_MODEL_ID = "" #@param {type:"string"}

print(f"✓ Ready" if ISTARI_PAT else "✗ PAT not found (check Colab Secrets)")

In [ ]:
#@title Wing Design Parameters { display-mode: "form" }

Length_Overall_in = 99.9 #@param {type:"number"}
Wingspan_in = 144.0 #@param {type:"number"}
Leading_Edge_Sweep_Inboard_deg = 46.5 #@param {type:"number"}
Leading_Edge_Sweep_Outboard_deg = 46.5 #@param {type:"number"}
Trailing_Edge_Sweep_Inboard_deg = -46.5 #@param {type:"number"}
Trailing_Edge_Sweep_Outboard_deg = 15.0 #@param {type:"number"}
Panel_Break_Span_Fraction = 0.3 #@param {type:"number"}

# Map to internal variable names
loa_in = Length_Overall_in
span = Wingspan_in
le_sweep_p1 = Leading_Edge_Sweep_Inboard_deg
le_sweep_p2 = Leading_Edge_Sweep_Outboard_deg
te_sweep_p1 = Trailing_Edge_Sweep_Inboard_deg
te_sweep_p2 = Trailing_Edge_Sweep_Outboard_deg
panel_break_span_pct = Panel_Break_Span_Fraction

In [ ]:
#@title Analyze and Verify { display-mode: "form" }
import json
import time
import logging
from istari_digital_client.client import Client
from istari_digital_client.configuration import Configuration
from istari_digital_client import JobStatusName
from istari_digital_client.exceptions import (
    BadRequestException,
    ForbiddenException,
    NotFoundException,
    ApiException,
)

def run_analysis():
    """Main analysis function - returns True on success, False on failure."""
    
    # === Material Cost Estimates ($/lb) ===
    COMPOSITE_COST_PER_LB = 85.00  # Carbon fiber composite
    METAL_COST_PER_LB = 12.50      # Aluminum alloy

    # === Helper for friendly errors ===
    def handle_api_error(e, context="API call"):
        """Convert API exceptions to user-friendly messages."""
        if isinstance(e, BadRequestException):
            if "Invalid Personal Access Token" in str(e):
                print("\n❌ INVALID PERSONAL ACCESS TOKEN")
                print("   Your ISTARI_PAT is invalid or expired.")
                print("   → Check Colab Secrets (🔑 in sidebar)")
                print("   → Verify the token is correct and not expired")
            else:
                print(f"\n❌ BAD REQUEST: {e.body if hasattr(e, 'body') else e}")
        elif isinstance(e, ForbiddenException):
            print("\n❌ ACCESS DENIED")
            print("   You don't have permission to access this resource.")
            print("   → Check that your PAT has the required permissions")
            print("   → Verify you have access to the model")
        elif isinstance(e, NotFoundException):
            print("\n❌ NOT FOUND")
            print("   The requested resource was not found.")
            print("   → Check that the NTOP_MODEL_ID is correct")
        elif isinstance(e, ApiException):
            print(f"\n❌ API ERROR ({e.status if hasattr(e, 'status') else 'unknown'})")
            print(f"   {e.body if hasattr(e, 'body') else e}")
        else:
            print(f"\n❌ ERROR during {context}")
            print(f"   {e}")

    # === Validate inputs ===
    if not ISTARI_PAT:
        print("❌ ISTARI_PAT not found!")
        print("   → Click the 🔑 icon in the Colab sidebar")
        print("   → Add a secret named 'ISTARI_PAT' with your token")
        return False

    if not NTOP_MODEL_ID:
        print("❌ NTOP_MODEL_ID is empty!")
        print("   → Enter a valid model ID in the User Data section")
        return False

    # === Connect to Istari ===
    print("Connecting to Istari...")
    logging.getLogger('istari_digital_client').setLevel(logging.CRITICAL)

    try:
        client = Client(
            config=Configuration(
                registry_url=ISTARI_ENVIRONMENT_URL,
                registry_auth_token=ISTARI_PAT,
            )
        )
        ntop_model = client.get_model(NTOP_MODEL_ID)
        print(f"✓ Connected ({ntop_model.display_name or ntop_model.name})")
    except (BadRequestException, ForbiddenException, NotFoundException, ApiException) as e:
        handle_api_error(e, "connecting to Istari")
        return False
    except Exception as e:
        print(f"\n❌ CONNECTION ERROR: {e}")
        print("   → Check your internet connection")
        print("   → Verify ISTARI_ENVIRONMENT_URL is correct")
        return False

    # === Step 1: Build input parameters ===
    print(f"Configuring wing: LOA={loa_in}in, Span={span}in, LE Sweep={le_sweep_p1}°/{le_sweep_p2}°, TE Sweep={te_sweep_p1}°/{te_sweep_p2}°")

    input_json_data = {
        "inputs": [
            {"name": "LOA In", "type": "real", "units": "in", "value": loa_in},
            {"name": "Span", "type": "real", "units": "in", "value": span},
            {"name": "LE Sweep P1", "type": "real", "units": "deg", "value": le_sweep_p1},
            {"name": "LE Sweep P2", "type": "real", "units": "deg", "value": le_sweep_p2},
            {"name": "TE Sweep P1", "type": "real", "units": "deg", "value": te_sweep_p1},
            {"name": "TE Sweep P2", "type": "real", "units": "deg", "value": te_sweep_p2},
            {"name": "Panel Break Span %", "type": "real", "value": panel_break_span_pct},
            {"name": "MAIN PATH", "type": "file_path", "value": "/home/bradrothenberg/nTopGrp3/output/"}
        ]
    }

    # === Step 2: Run nTop analysis ===
    print("Starting nTop analysis...")

    start_time = time.time()

    try:
        run_job = client.add_job(
            model_id=ntop_model.id,
            function="@ntop:run_model",
            tool_name="ntopcl",
            tool_version="5.30",
            operating_system="RHEL 8",
            parameters={"ntop_input_json": input_json_data},
        )
    except (BadRequestException, ForbiddenException, ApiException) as e:
        handle_api_error(e, "starting analysis job")
        return False

    print(f"Job ID: {run_job.id}")
    print("-" * 40)

    last_status = None
    while run_job.status.name not in [JobStatusName.COMPLETED, JobStatusName.FAILED]:
        time.sleep(5)
        try:
            run_job = client.get_job(run_job.id)
        except ApiException as e:
            handle_api_error(e, "checking job status")
            return False
        
        if run_job.status.name != last_status:
            elapsed = time.time() - start_time
            status_name = run_job.status.name.name
            status_msg = status_name.replace("_", " ").lower()
            
            # Show elapsed time in friendly format
            if elapsed < 60:
                time_str = f"{elapsed:.0f}s"
            else:
                mins, secs = divmod(int(elapsed), 60)
                time_str = f"{mins}m {secs}s"
            
            print(f"  [{time_str}] {status_msg}...")
            last_status = run_job.status.name

    end_time = time.time()
    total_time = end_time - start_time

    logging.getLogger('istari_digital_client').setLevel(logging.WARNING)

    if run_job.status.name == JobStatusName.FAILED:
        print(f"\n❌ nTop analysis failed")
        print(f"   Job ID: {run_job.id}")
        print("   → The nTop model encountered an error during execution")
        print("   → Check the job logs in Istari for details")
        return False

    # Format total time
    if total_time < 60:
        total_str = f"{total_time:.0f} seconds"
    elif total_time < 3600:
        mins, secs = divmod(int(total_time), 60)
        total_str = f"{mins}m {secs}s"
    else:
        hrs, remainder = divmod(int(total_time), 3600)
        mins, secs = divmod(remainder, 60)
        total_str = f"{hrs}h {mins}m {secs}s"

    print(f"  [{total_str}] complete ✓")

    # === Step 3: Extract outputs ===
    print("\nExtracting results...")

    try:
        ntop_model = client.get_model(ntop_model.id)
    except ApiException as e:
        handle_api_error(e, "fetching results")
        return False

    output_values = {}
    for artifact in ntop_model.artifacts:
        if artifact.name == "output.json":
            try:
                output_data = json.loads(artifact.read_text())
                if isinstance(output_data, list):
                    for item in output_data:
                        if isinstance(item, dict) and item.get("type") == "json":
                            output_values = item.get("value", {}).get("jsonObject", {})
            except Exception as e:
                print(f"⚠️  Warning: Could not parse output.json: {e}")

    # Extract all available outputs
    weight_composite = output_values.get("Weight_Composite (lbm)")
    weight_metal = output_values.get("Weight_Metal (lbm)")
    wingtip_displacement = output_values.get("wingtipDisplacement (in)")
    volume = output_values.get("Volume (in^3)") or output_values.get("Volume")
    surface_area = output_values.get("Surface_Area (in^2)") or output_values.get("Surface_Area")

    # Validate we got results
    if weight_composite is None or weight_metal is None:
        print("\n⚠️  WARNING: Could not extract weight outputs from results")
        print(f"   Available outputs: {list(output_values.keys())}")

    # Calculate derived values
    total_weight = (weight_composite or 0) + (weight_metal or 0)
    composite_cost = (weight_composite or 0) * COMPOSITE_COST_PER_LB
    metal_cost = (weight_metal or 0) * METAL_COST_PER_LB
    total_material_cost = composite_cost + metal_cost

    # === Step 4: Verify requirements ===
    requirements = {
        "Total_Weight": {"max": 500.0, "units": "lb"},
        "Wingtip_Displacement": {"max": 0.5, "units": "in"}
    }

    weight_ok = total_weight > 0 and total_weight <= requirements["Total_Weight"]["max"]
    displacement_ok = wingtip_displacement is not None and wingtip_displacement <= requirements["Wingtip_Displacement"]["max"]
    all_pass = weight_ok and displacement_ok

    # === Display Results ===
    print("\n" + "=" * 55)
    print("               ANALYSIS COMPLETE")
    print("=" * 55)

    print("\n  STRUCTURAL PROPERTIES")
    print("  " + "-" * 40)
    if weight_composite is not None:
        print(f"    Composite Weight:    {weight_composite:>10.2f} lb")
    if weight_metal is not None:
        print(f"    Metal Weight:        {weight_metal:>10.2f} lb")
    print(f"    Total Weight:        {total_weight:>10.2f} lb  {'✅' if weight_ok else '❌'}")
    if wingtip_displacement is not None:
        print(f"    Wingtip Deflection:  {wingtip_displacement:>10.4f} in  {'✅' if displacement_ok else '❌'}")

    if volume is not None:
        print(f"    Volume:              {volume:>10.2f} in³")
    if surface_area is not None:
        print(f"    Surface Area:        {surface_area:>10.2f} in²")

    print("\n  MATERIAL COST ESTIMATE")
    print("  " + "-" * 40)
    print(f"    Composite (${COMPOSITE_COST_PER_LB:.0f}/lb): ${composite_cost:>10,.2f}")
    print(f"    Metal (${METAL_COST_PER_LB:.0f}/lb):      ${metal_cost:>10,.2f}")
    print(f"    Total Material Cost:   ${total_material_cost:>10,.2f}")

    print("\n  JOB INFO")
    print("  " + "-" * 40)
    print(f"    Job ID: {run_job.id}")
    print(f"    Total Time: {total_str}")

    print("\n" + "=" * 55)
    if all_pass:
        print("  ✅ ALL REQUIREMENTS PASSED!")
        print(f"  🎯 Structure Weight: {total_weight:.2f} lb | Cost: ${total_material_cost:,.2f}")
        print("     Can you optimize further?")
    else:
        print("  ❌ REQUIREMENTS NOT MET")
        if not weight_ok:
            print(f"     - Weight {total_weight:.1f} lb exceeds {requirements['Total_Weight']['max']} lb max")
        if not displacement_ok and wingtip_displacement is not None:
            print(f"     - Deflection {wingtip_displacement:.4f} in exceeds {requirements['Wingtip_Displacement']['max']} in max")
    print("=" * 55)
    
    return True

# Run the analysis
run_analysis()